In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras_tuner
from dataclasses import dataclass
from typing import List
from keras import layers, ops
from keras.utils import timeseries_dataset_from_array
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

from processing import split_indices, make_stationary, inverse_stationary, scale, unscale, sequence, Stationarizer, Normalizer
from tuner import LSTMLayerHyperparams, ModelHyperparams
from plotting import visualize_loss, show_plot
from model_builder import get_model, build_model, get_tuner

In [9]:
data = pd.read_csv("data/YF_AAPL.csv")["Adj Close"].to_numpy().reshape(-1, 1)

train_idx = int(0.7 * np.shape(data)[0])
val_idx = int(0.85 * np.shape(data)[0])

train_data = data[:train_idx]
val_data = data[train_idx:val_idx]
test_data = data[val_idx:]

In [10]:
train_stater = Stationarizer()
val_stater = Stationarizer()
normer = Normalizer()
train_data = train_stater.fit_transform(train_data)
val_data = val_stater.fit_transform(val_data)
train_data = normer.fit_transform(train_data)
val_data = normer.transform(val_data)

In [11]:
train_ds = timeseries_dataset_from_array(
    train_data[:-20],
    train_data[20:],
    sequence_length=20,
    batch_size=32,
    shuffle=True
)
val_ds = timeseries_dataset_from_array(
    val_data[:-20],
    val_data[20:],
    sequence_length=20,
    batch_size=32,
    shuffle=False
)

for batch in train_ds.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (32, 20, 1)
Target shape: (32, 1)


2024-08-02 15:30:39.289927: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
tuner = get_tuner()
tuner.search(train_ds, epochs=50, validation_data=val_ds)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
96                |96                |units_0
tanh              |tanh              |activation_0
sigmoid           |sigmoid           |recurrent_activation_0
0.5               |0.5               |dropout_0
0.4               |0.4               |recurrent_dropout_0
sigmoid           |sigmoid           |activation_1
0.014727          |0.014727          |lr

Epoch 1/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 1.0223 - mse: 1.0223 - val_loss: 45.9369 - val_mse: 45.9369
Epoch 2/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.9924 - mse: 0.9924 - val_loss: 45.9369 - val_mse: 45.9369
Epoch 3/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.9988 - mse: 0.9988 - val_loss: 45.9369 - val_mse: 45.9369
Epoch 4/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.9897 - mse: 0.9897 - val_loss: 45.9369 - val_mse: 45.9369
Epoch 5/50
134/134 ━━━━━━━━━━━

KeyboardInterrupt: 